# Question Answering with Transformers

Question Answering (QA) is a field within natural language processing (NLP) that involves building systems capable of answering questions posed by humans in natural language. It's a challenging area that combines understanding of natural language semantics with the ability to retrieve, process, and extract relevant information from data sources.

### Components of QA Systems

Building an effective QA system involves several components and stages:

- Question Processing: This involves understanding and analyzing the question to determine its type and the information it seeks. This might include parsing the question, identifying key terms, and sometimes transforming the question into a form more amenable to computation.
- Document or Data Retrieval: For many QA tasks, especially those that are open-domain, the system needs to retrieve data from which the answer can be extracted. This could involve querying a structured database or searching through unstructured text data.
- Information Processing: Once the relevant information is retrieved, it must be processed. For unstructured data, this often involves natural language understanding tasks such as named entity recognition, part-of-speech tagging, and dependency parsing.
- Answer Extraction: The final step is to extract or generate an answer from the processed data. In some systems, this might also involve generating natural language responses based on the data.

### Transformer Models in QA
Transformers have revolutionized the field of NLP, and they are particularly effective in QA tasks due to their ability to handle context and their deep understanding of language semantics. With their attention mechanisms, transformers learn complex relationships between words in both the question and the context (the text from which we want to extract the answer).  Models like BERT (Bidirectional Encoder Representations from Transformers) have been pre-trained on large text corpora using tasks like masked language modeling, which helps them understand context and the relationships between words.

For QA, transformers are typically fine-tuned on a QA dataset like the Stanford Question Answering Dataset (SQuAD), where they learn to predict the start and end positions of answers within a given passage of text. The fine-tuning process adjusts the model's weights to better perform this specific task, leveraging the general capabilities learned during pre-training.



### Example 1: QA example using BERT Model


In [1]:
import torch
from transformers import BertTokenizer, BertForQuestionAnswering

The example below works in the following way:
1. Initialization: We load a pre-trained BERT model specifically fine-tuned for the question answering task (SQuAD dataset).
2. Tokenization: The question and context texts are tokenized in a way that BERT expects, with special tokens added and both texts combined into a single sequence.
3. Model Prediction: The model provides two sets of scores for each token—start and end scores indicating the likelihood of a token being the start or end of the answer.
4. Determine Answer: The tokens with the highest start and end scores are identified, and the corresponding part of the text is extracted as the answer.



In [3]:
# Load a pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# Define the question and context
question = "What is the capital of France?"
context = "France is a country in Europe. Its capital is Paris."

# Encode the question and context so that they are prepared for the model
inputs = tokenizer.encode_plus(question, context, return_tensors='pt', add_special_tokens=True)

# Get model's prediction
input_ids = inputs['input_ids']
token_type_ids = inputs['token_type_ids']

# Model output handling might require adjustment based on transformers version
outputs = model(input_ids, token_type_ids=token_type_ids)
start_scores = outputs.start_logits
end_scores = outputs.end_logits

# Find the position tokens with the highest scores
start_index = torch.argmax(start_scores)
end_index = torch.argmax(end_scores) + 1

# Convert tokens to the answer string
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][start_index:end_index]))

print("Answer:", answer)


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Answer: paris


### Example 2: QA Pipeline in Hugging Face Transformers

The Hugging Face `transformers` library provides a convenient and powerful way to utilize state-of-the-art NLP models for various tasks, including question answering (QA). One of the key features of this library is the [**pipeline**](https://huggingface.co/docs/transformers/en/main_classes/pipelines) function, which abstracts much of the complexity involved in processing data and making predictions. The next example illustrates the question answering pipeline, detailing its functionality and usage.

In [ ]:
from transformers import pipeline

In [6]:
# create pieline for QA
qa = pipeline('question-answering')

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


**Components of the QA Pipeline**
- Model (optional paramter): By default, the QA pipeline uses a transformer model pre-trained and fine-tuned on a question answering dataset. Models like` bert-large-uncased-whole-word-masking-finetuned-squad` or `distilbert-base-uncased-distilled-squad` are commonly used, but the pipeline can be configured to use any compatible model from the Hugging Face Model Hub.
- Tokenizer: The tokenizer appropriate to the model’s architecture is used to properly format the input text.

In [11]:
question = "What is the capital of France?"
context = "France is a country in Europe. Its capital is Paris."
qa(context = context, question = question)

{'score': 0.984136164188385, 'start': 46, 'end': 51, 'answer': 'Paris'}

In [16]:
question = "Who is Elon Musk?"
context = '''Elon Reeve Musk is a business magnate and investor. He is the founder, CEO and chief engineer of SpaceX; 
            angel investor, CEO and product architect of Tesla, Inc.; owner and CEO of Twitter, Inc.; founder of The Boring Company; co-founder of Neuralink 
            and OpenAI; and president of the philanthropic Musk Foundation. With an estimated net worth of around $175 billion as of February 3, 2023, primarily 
            from his ownership stakes in Tesla and SpaceX,[4][5] Musk is the second-wealthiest person in the world, according to both the Bloomberg Billionaires 
            Index and Forbes's real-time billionaires list.[6][7]
            Musk was born in Pretoria, South Africa, and briefly attended at the University of Pretoria before moving to Canada at age 18, acquiring citizenship 
            through his Canadian-born mother. Two years later, he matriculated at Queen's University and transferred to the University of Pennsylvania, 
            where he received bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University. After two days, he 
            dropped out and with his brother Kimbal, co-founded the online city guide software company Zip2. In 1999, Zip2 was acquired by Compaq for $307 million 
            and Musk co-founded X.com, a direct bank. X.com merged with Confinity in 2000 to form PayPal, which eBay acquired for $1.5 billion in 2002.
            With $175.8 million, Musk founded SpaceX in 2002, a spaceflight services company. In 2004, he was an early investor in the electric vehicle manufacturer 
            Tesla Motors, Inc. (now Tesla, Inc.). He became its chairman and product architect, assuming the position of CEO in 2008. In 2006, he helped create 
            SolarCity, a solar energy company that was later acquired by Tesla and became Tesla Energy. In 2015, he co-founded OpenAI, a nonprofit artificial 
            intelligence research company. The following year, he co-founded Neuralink—a neurotechnology company developing brain–computer interfaces—and The Boring 
            Company, a tunnel construction company. Musk has also proposed a hyperloop high-speed vactrain transportation system. In 2022, his acquisition of Twitter 
            for $44 billion was completed.'''
qa(context = context, question = question)

{'score': 0.5220692157745361,
 'start': 21,
 'end': 50,
 'answer': 'business magnate and investor'}

The `score` is a numerical value between 0 and 1 that indicates the confidence level of the model in the correctness of the provided answer. 

In [6]:
question = "What companies has Elon Musk invested in?"
qa(context = context, question = question)

{'score': 0.9309016466140747,
 'start': 467,
 'end': 483,
 'answer': 'Tesla and SpaceX'}

### Limitations of QA Systems

QA models, especially those built on transformer architectures like BERT, are powerful tools for extracting answers from given text. However, their accuracy and reliability are heavily dependent on the relevance and completeness of the context provided with the question.

In [8]:
question = "What is the height of Mount Everest?"
context = "Mount Everest is located in the Mahalangur Himal sub-range of the Himalayas. The border between Nepal and China runs across its summit point."

# Use the pipeline to find the answer
result = qa({
    'question': question,
    'context': context
})

print(result)

{'score': 0.15850915014743805, 'start': 32, 'end': 75, 'answer': 'Mahalangur Himal sub-range of the Himalayas'}


As can be seen from the above example, when the answer to a question is not present in the provided text, the model can make incorrect or irrelevant predictions. Users must ensure that the context provided contains the information necessary to answer the question or implement additional checks to handle cases where the model's predictions might be based on incomplete or irrelevant data.